# This is only used for evaluation. Training is not yet supported!

In [1]:
from sae_lens.training.sparse_autoencoder import SparseAutoencoder
from sae_lens.training.config import LanguageModelSAERunnerConfig
from sae_lens.training.sae_group import SAEGroup
from huggingface_hub import hf_hub_download
from sparse_autoencoder import SparseAutoencoder

In [2]:
hf_model_id = "HuFY-dev/normalized_sparse_autoencoder"
# hf_filename = "baseline.pt"
hf_filename = "normalized.pt"
model_path = hf_hub_download(hf_model_id, hf_filename)

In [3]:
sae = SparseAutoencoder.load(model_path)

In [4]:
if sae.config.sae_type=="normalized_sae":
    sae_type = "tanh_sae"
else:
    sae_type = "unit_norm_sae"
sae_lens_configs = LanguageModelSAERunnerConfig(
        d_in=sae.config.n_input_features,
        d_sae=sae.config.n_learned_features,
        expansion_factor=sae.config.n_learned_features//sae.config.n_input_features,
        sae_type=sae_type,
        noise_scale=sae.config.noise_scale,
        model_name="gpt2",
        hook_point="blocks.{layer}.hook_mlp_out",
        hook_point_layer=list(range(sae.config.n_components)),  # type: ignore
    )

sae_group = SAEGroup(sae_lens_configs)

for layer, single_sae in enumerate(sae_group.autoencoders):
    single_sae.W_enc.data = sae.encoder.weight.data[layer].T
    single_sae.b_enc.data = sae.encoder.bias.data[layer]
    single_sae.W_dec.data = sae.decoder.weight.data[layer].T
    single_sae.b_dec.data = sae.tied_bias.data[layer]

Run name: 12288-L1-0.001-LR-0.0003-Tokens-2.000e+06
n_tokens_per_buffer (millions): 0.08192
Lower bound: n_contexts_per_buffer (millions): 0.00064
Total training steps: 488
Total wandb updates: 48
n_tokens_per_feature_sampling_window (millions): 1048.576
n_tokens_per_dead_feature_window (millions): 524.288
We will reset the sparsity calculation 0 times.
Number tokens in sparsity calculation window: 8.19e+06
Run name: 12288-L1-0.001-LR-0.0003-Tokens-2.000e+06
n_tokens_per_buffer (millions): 0.08192
Lower bound: n_contexts_per_buffer (millions): 0.00064
Total training steps: 488
Total wandb updates: 48
n_tokens_per_feature_sampling_window (millions): 1048.576
n_tokens_per_dead_feature_window (millions): 524.288
We will reset the sparsity calculation 0 times.
Number tokens in sparsity calculation window: 8.19e+06
Run name: 12288-L1-0.001-LR-0.0003-Tokens-2.000e+06
n_tokens_per_buffer (millions): 0.08192
Lower bound: n_contexts_per_buffer (millions): 0.00064
Total training steps: 488
Total

In [5]:
# Now you should be able to use sae_group
sae_group

# Small sanity check

In [6]:
from typing import TypedDict
from transformer_lens import HookedTransformer
from datasets import load_dataset
import torch
from torch.utils.data import DataLoader

In [7]:
model_name = "gpt2"
model = HookedTransformer.from_pretrained(model_name)

Loaded pretrained model gpt2 into HookedTransformer


In [8]:
dataset_path = "alancooney/sae-monology-pile-uncopyrighted-tokenizer-gpt2"
torch_dataset = load_dataset(dataset_path, split="train", streaming=True).with_format("torch")

Resolving data files:   0%|          | 0/64 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/64 [00:00<?, ?it/s]

In [9]:
TokenizedPrompt = list[int]
"""A tokenized prompt."""


class TokenizedPrompts(TypedDict):
    """Tokenized prompts."""

    input_ids: list[TokenizedPrompt]
    
class TorchTokenizedPrompts(TypedDict):
    """Tokenized prompts prepared for PyTorch."""

    input_ids: torch.Tensor

dl = DataLoader[TorchTokenizedPrompts](
            torch_dataset,
            batch_size=16,
            # Shuffle is most efficiently done with the `shuffle` method on the dataset itself, not
            # here.
            shuffle=False,
            num_workers=1,
        )

In [10]:
hooked_layers = [sae_group.cfg.hook_point.format(layer=layer) for layer in sae_group.cfg.hook_point_layer]
hooked_layers

['blocks.0.hook_mlp_out',
 'blocks.1.hook_mlp_out',
 'blocks.2.hook_mlp_out',
 'blocks.3.hook_mlp_out',
 'blocks.4.hook_mlp_out',
 'blocks.5.hook_mlp_out',
 'blocks.6.hook_mlp_out',
 'blocks.7.hook_mlp_out',
 'blocks.8.hook_mlp_out',
 'blocks.9.hook_mlp_out',
 'blocks.10.hook_mlp_out',
 'blocks.11.hook_mlp_out']

In [11]:
residuals = []
for i, batch in enumerate(dl):
    if i >= 1:
        break
    batch_tokens = batch["input_ids"]
    _, cache = model.run_with_cache(batch_tokens, prepend_bos=True, names_filter=hooked_layers)
    residuals = [cache[layer] for layer in hooked_layers]
    del cache

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [12]:
sae_hooks = ["hook_hidden_post", "hook_sae_out"]
for i in range(len(residuals)):
    autoencoder = sae_group.autoencoders[i]
    _, cache = autoencoder.run_with_cache(residuals[i].to(autoencoder.device), names_filter=sae_hooks)
    reconstructed = cache["hook_sae_out"]
    feature_act = cache["hook_hidden_post"]
    l2_loss = torch.nn.functional.mse_loss(residuals[i].to(autoencoder.device), reconstructed)
    l1_loss = torch.nn.functional.l1_loss(feature_act, torch.zeros_like(feature_act)) * autoencoder.d_sae
    print(f"Layer {i}: L2 loss: {l2_loss}, L1 loss: {l1_loss}")
    del cache

Layer 0: L2 loss: 0.04833340272307396, L1 loss: 77.42855834960938
Layer 1: L2 loss: 0.04231907054781914, L1 loss: 172.45379638671875
Layer 2: L2 loss: 0.06545423716306686, L1 loss: 128.70123291015625
Layer 3: L2 loss: 0.04924310743808746, L1 loss: 186.74769592285156
Layer 4: L2 loss: 0.05221531167626381, L1 loss: 207.7931671142578
Layer 5: L2 loss: 0.06078438460826874, L1 loss: 221.4296875
Layer 6: L2 loss: 0.07589776068925858, L1 loss: 241.465576171875
Layer 7: L2 loss: 0.10232851654291153, L1 loss: 245.15988159179688
Layer 8: L2 loss: 0.13378532230854034, L1 loss: 249.77183532714844
Layer 9: L2 loss: 0.20779462158679962, L1 loss: 234.27230834960938
Layer 10: L2 loss: 0.5099189281463623, L1 loss: 141.11851501464844
Layer 11: L2 loss: 0.892001211643219, L1 loss: 117.59300994873047


Pretty similar to the results I got.